PART 2 - DATA EXPLORATION

In [1]:
import pandas as pd

df = pd.read_csv("unesco_train.csv")
df.set_index(["Country", "Year"], drop=False, inplace=True)
df.head()

Country  Year  fertilityRate  lifeExpectancy  mortalityRate  \
Country   Year                                                                  
Australia 2015  Australia  2015          1.814        82.40000            3.2   
          2016  Australia  2016          1.752        82.44878            3.1   
          2017  Australia  2017          1.765        82.49756            3.0   
Austria   2015    Austria  2015          1.490        81.19024            3.0   
          2016    Austria  2016          1.530        81.64146            3.0   

                popGrowth  ruralPopPct  gdpGrowthPct         gdpUS  \
Country   Year                                                       
Australia 2015    1.43922       14.299       2.35114  1.349034e+12   
          2016    1.56194       14.200       2.82731  1.208039e+12   
          2017    1.68452       14.096       1.95758  1.323421e+12   
Austria   2015    1.12099       42.285       1.14298  3.818057e+11   
          2016    1.08140       42.095       2.03957  3.940528e+11   

                gdpPerCapitaUS  ...  gniPerCapita  pop14under  pop15to24  \
Country   Year                  ...                                        
Australia 2015     56644.03396  ...       60440.0    4520.213   3172.395   
          2016     49937.73139  ...       54180.0    4598.344   3155.240   
          2017     53793.53726  ...       51360.0    4687.067   3135.675   
Austria   2015     44176.67174  ...       47490.0    1225.425   1008.434   
          2016     45103.32981  ...       46130.0    1239.550   1001.790   

                pop25to64  pop65over   totalPop  gdpPerCapita ppp  \
Country   Year                                                      
Australia 2015  12684.814   3555.080  23932.502               NaN   
          2016  12835.482   3673.646  24262.712               NaN   
          2017  12974.858   3787.020  24584.620               NaN   
Austria   2015   4809.700   1635.101   8678.660               NaN   
          2016   4851.921   1654.040   8747.301               NaN   

                gdp pppInternational  gniPerCapita ppp  happinessScore  
Country   Year                                                          
Australia 2015                   NaN               NaN           7.284  
          2016                   NaN               NaN           7.313  
          2017                   NaN               NaN           7.284  
Austria   2015                   NaN               NaN           7.200  
          2016                   NaN               NaN           7.119  

[5 rows x 21 columns]

First, I explored the most happy and least happy countries for years 2015-2017 and wanted to see if these countries would match my intuition.

In [2]:
df_2015 = df[df["Year"] == 2015]
df_2016 = df[df["Year"] == 2016]
df_2017 = df[df["Year"] == 2017]

df_yr_dict = {2015: df_2015, 2016: df_2016, 2017: df_2017}
for k, v in df_yr_dict.items():
    print("\nTop 3 'Most Happy' Countries (%d)" % k)
    display(v["happinessScore"].sort_values(ascending=False)[:3])
    print("\nTop 3 'Least Happy' Countries (%d)" % k)
    display(v["happinessScore"].sort_values(ascending=True)[:3])



Top 3 'Most Happy' Countries (2015)


Country      Year
Switzerland  2015    7.587
Iceland      2015    7.561
Denmark      2015    7.527
Name: happinessScore, dtype: float64


Top 3 'Least Happy' Countries (2015)


Country  Year
Togo     2015    2.839
Burundi  2015    2.905
Syria    2015    3.006
Name: happinessScore, dtype: float64


Top 3 'Most Happy' Countries (2016)


...

Country  Year
Burundi  2016    2.905
Syria    2016    3.069
Togo     2016    3.303
Name: happinessScore, dtype: float64


Top 3 'Most Happy' Countries (2017)


Country  Year
Norway   2017    7.537
Denmark  2017    7.522
Iceland  2017    7.504
Name: happinessScore, dtype: float64


Top 3 'Least Happy' Countries (2017)


Country                   Year
Central African Republic  2017    2.693
Burundi                   2017    2.905
Tanzania                  2017    3.349
Name: happinessScore, dtype: float64

The results above match my intuition.  
- Switzerland is recorded as one of the top 3 happiest countries in 2015 and 2016 and this makes sense considering Switzerland is known for their excellent health care, family planning, and low death rates.
- Syria, on the other hand, is recorded as one of the bottom 3 hhappiest countries in 2015 and 2016.  This makes sense considering Syria was (and still is) unergoing a civil war. 

Now, I will train this data, using the happinessScore as the labels, to fit machine learning regression models.  I will use the models I have fit to predict on the larger unesco data set (years 1970-2019).  The rest of my exploration will inlove this larger dataset and my corresponding predictions for it.